# Below is to prepare and cleanup the collected data from EJFAT and ERSAP for Digital Twin. 

In [ ]:
import pandas as pd
import glob
from matplotlib import pyplot as plt
import os

os.chdir("/workspaces/Queue_Simulation_Python/ejfat-data")

In [ ]:
# remove outlier data points based on the z-score
def remove_outliers(df, columns, threshold=1):
    from scipy import stats
    import numpy as np

    z = np.abs(stats.zscore(df[columns]))
    outlier_indices = (z > threshold).any(axis=1)
    return df[outlier_indices].index


In [ ]:
dfs_event_rate = []
dfs_process_time = []
dfs_drop_total_event_rate = []

for folder in glob.glob("*/"):
    fs = glob.glob(f"{folder}/*.csv")
    folder = folder.replace("dt-vol-", "").replace("/", "")
    for f in fs:
        df = pd.read_csv(f)
        if "currDropTotalEvents" in f:
            df.columns = ["time", "current-drop-total-event-rate"]
            # add a column of folder
                # remove "dt-vol" in folder name
            df["folder"] = folder
            dfs_drop_total_event_rate.append(df)

        elif "Process Time" in f:
            df.columns = ["time", "process-time"]
            # the entry is like "1.22 ms", and remove the "ms" and convert to float
            df["process-time"] = df["process-time"].str.replace("ms", "").astype(float)
            title = "process-time"
            df["folder"] = folder
            dfs_process_time.append(df)

        elif "evRate" in f:
            df.columns = ["time", "event-rate"]
            title = "event-rate"
            df["folder"] = folder
            dfs_event_rate.append(df)

dfs_event_rate = pd.concat(dfs_event_rate, ignore_index=True)
dfs_process_time = pd.concat(dfs_process_time, ignore_index=True)
dfs_drop_total_event_rate = pd.concat(dfs_drop_total_event_rate, ignore_index=True)

# remove outliers
outliers = remove_outliers(dfs_process_time, ["process-time"], 5)
print(f"outliers: {outliers}")
## remove the rows with outliers
dfs_process_time = dfs_process_time.drop(outliers)
dfs_event_rate = dfs_event_rate.drop(outliers)
dfs_drop_total_event_rate = dfs_drop_total_event_rate.drop(outliers)

print(f"type of columns: {dfs_event_rate.dtypes}")

display(dfs_event_rate)
display(dfs_process_time)
display(dfs_drop_total_event_rate)

In [ ]:
# plot process time
plt.figure()
for folder, df in dfs_process_time.groupby("folder"):
    plt.plot(df["time"], df["process-time"], label=folder)
plt.legend()
plt.title("Process Time")
plt.xlabel("time")
plt.ylabel("process time (ms)")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(10, 15))

dataframes = [dfs_event_rate, dfs_process_time, dfs_drop_total_event_rate]
for idx, df in enumerate(dataframes):
    # get the mean and std of each folder
    df_stat = df.groupby("folder").describe()[df.columns[1]].sort_values("mean")

    df_mean = df_stat["mean"]
    df_std = df_stat["std"]
    df_count = df_stat["count"]

    # plot mean and std of each folder in the same plot. The dot is the mean, and the error bar is the std centered at the mean
    axs[idx].errorbar(df_mean.index, df_mean, yerr=df_std, fmt='o')
    # annotate the mean and percentage of the std
    for i, (mean, std) in enumerate(zip(df_mean, df_std)):
        axs[idx].annotate(f"{mean:.2f}±{std:.2f}", (i, mean))
        
        
    axs[idx].set_xlabel("test")
    # make xlabel vertical
    axs[idx].set_xticklabels(df_mean.index, rotation=30)
    if "process-time" in df.columns[1]:
        axs[idx].set_ylabel("process time (ms)")
    else:
        axs[idx].set_ylabel(df.columns[1]+" (Hz)")
    axs[idx].set_title(df.columns[1])

    # plot the number of data points in each folder as a additional subplot
    ax2 = axs[idx].twinx()
    ax2.set_ylabel("number of data points")
    ax2.bar(df_mean.index, df_count.loc[df_mean.index], alpha=0.2)

plt.tight_layout()

plt.show()

# Below is for calculating Quantity of interest

In [ ]:
# define lamba as dfs_event_rate["event-rate"]
ls = dfs_event_rate.groupby("folder").apply(lambda x: x["event-rate"].mean())
## add column name lambda
ls = ls.reset_index()
ls.columns = ["folder", "lambda"]
ls = ls.set_index("folder")
# print(f"ls: {ls}")


# define mu as dfs_process_time["process-time"]
mus = dfs_process_time.groupby("folder").apply(lambda x: 1/(x["process-time"].mean()* 1e-3))
# print(f"mus: {mus}") 
# add mus as a new column in ls by matching the folder name
ls.loc[mus.index, "mu"] = mus.values
# print(f"ls: {ls}")


# add a column of server number in ls
ls = ls.reset_index()
ls["server-number"] = ls["folder"].apply(lambda x: int(x.split("-")[1][0]))
ls = ls.set_index("folder")
# print(f"ls: {ls}")


# calculate the utilization
ls["utilization"] = ls["lambda"] / ls["mu"] / ls["server-number"]

# round ls to 2 decimal places
ls = ls.round(2)

display(ls)


# Get Lq from current total drop events.

In [ ]:
test_df = dfs_drop_total_event_rate.loc[dfs_drop_total_event_rate["folder"] == "1g-1node"]

In [ ]:
dfs_drop_total_events = dfs_drop_total_event_rate.copy()
dfs_drop_total_events["drop-total-events"] = dfs_drop_total_events.groupby("folder")["current-drop-total-event-rate"].cumsum()
dfs_drop_total_events = dfs_drop_total_events.groupby("folder").last()
dfs_drop_total_events = dfs_drop_total_events.drop("current-drop-total-event-rate", axis=1)
# drop time column
dfs_drop_total_events = dfs_drop_total_events.drop("time", axis=1)
# 
display(dfs_drop_total_events)

